## Conexión a la API de Twitter

Para la ejecución de este código se requiere tener el fichero `twitter_credentials.json`
con los credenciales generados para la aplicación.

Comenzaremos importando las librerías que nos harán falta a posteriori.
El código aquí recogido en bloques también se puede encontrar como script de
`Python` [aquí](_legacy/twitter_raw.py).

In [3]:
import json
import requests

from enum import Enum
from typing import Union, Optional

from authlib.integrations.requests_client import (
    OAuth1Auth,
    OAuth2Auth,
)

from urllib.parse import urlencode

Ahora podemos guardat las URLs a las que atacaremos con `requests`, en
concreto `https://api.twitter.com/2` es la URL base, a la cual le iremos
añadiendo los diferentes *end-points* documentados en la
[web oficial](https://developer.twitter.com/en/docs/twitter-api),
además creamos una sesión permanente:

In [10]:
base_api_url_v2 = "https://api.twitter.com/2"
tweets_url_v2 = f"{base_api_url_v2}/tweets"
session = requests.session()

A continuación procedemos con la creación del credencial OAuth que usaremos.
Para ello definimos las siguientes funciones:

In [6]:
def load_credentials(file_path: str) -> dict:
    """Load credentials from json file
    :param file_path: path to the JSON file where the credentials are securely stored
    :return: Dict with the credential details with the following structure:
        {
          "CONSUMER_KEY": "YOUR_APP_KEY",
          "CONSUMER_SECRET": "YOUR_APP_SECRET_KEY",
          "ACCESS_TOKEN": "YOUR_ACCESS_TOKEN",
          "ACCESS_SECRET": "YOUR_ACCESS_SECRET_TOKEN"
        }
    """
    with open(file_path, "r") as file:
        creds = json.load(file)
    return creds


def get_app_auth(access_token: str) -> OAuth2Auth:
    """It instantiates an OAuth2Auth object given the credentials passed in `args`

    :param access_token: The access token required

    :return: OAuth2Auth object
    """
    r = OAuth2Auth(
        token={
            "access_token": access_token,
            "token_type": "Bearer"},
    )
    return r


def get_user_auth(
        consumer_key: str,
        consumer_secret: str,
        access_token: str,
        access_secret: str,
) -> OAuth1Auth:
    """It instantiates an OAuth1Auth object given the credentials passed in `args`

    :param consumer_key: The API_KEY generated for our twitter app
    :param consumer_secret: The API_SECRET generated for our twitter app
    :param access_token: The ACCESS_TOKEN generated for our twitter app
    :param access_secret: The ACCESS_SECRET generated for our twitter app

    :return: OAuth1Auth object
    """
    r = OAuth1Auth(
        client_id=consumer_key,
        client_secret=consumer_secret,
        token=access_token,
        token_secret=access_secret,
    )
    return r

In [8]:
# credenciales:
credentials = load_credentials(file_path="auth/twitter_credentials.json")

# app auth:
app_auth = get_app_auth(access_token=credentials["BEARER_TOKEN"])

# user auth:
user_auth = get_user_auth(
    consumer_key=credentials["CONSUMER_KEY"],
    consumer_secret=credentials["CONSUMER_SECRET"],
    access_token=credentials["ACCESS_TOKEN"],
    access_secret=credentials["ACCESS_SECRET"],
)

Una vez tenemos todos los ingredientes, podemos proceder con nuestra primera
consulta. Por ejemplo, podemos pedir nuestro identificador de usuario:

In [15]:
user_name = "madolivencia"

url = f"{base_api_url_v2}/users/by/username/{user_name}"

result = session.get(
    url=url,
    auth=user_auth
)
print(f"status code: {result.status_code}")
print(f"result: {result.json()}")

status code: 200
result = {'data': {'id': '1480491463410700289', 'name': 'Miguel Duran', 'username': 'madolivencia'}}


Como puedes observar, el resultado de la petición indica que la petición ha
sido procesada correctamente (`status_code = 200`) y que la respuesta correspondiente
está encapsulada en un diccionario que contiene el `id`, `nombre` y `username`.
Enhorabuena! Ya hemos hecho nuestra primera petición con éxito 🥳

Ahora será interesante que te familiarices con algunos el [modelo de datos de
Twitter](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/introduction), así como los principales *end points* para extraer información.
En el bloque siguiente puedes encontrar algunas definiciones que te pueden ser
útil para seguir experimentando (consultar la [documentación](https://developer.twitter.com/en/docs/twitter-api)
para entender como acceder a otros datos):

In [48]:
def get_tweets(ids: list, auth: Union[OAuth1Auth, OAuth2Auth]):
    """Gets tweets by their Ids

    :param ids: List of Ids of the tweets to be retrieved
    :param auth: OAuth object

    :return: Dict with the results
    """
    query = {"ids": ",".join(ids)}
    search = session.request(
        url=f"{tweets_url_v2}?{urlencode(query)}",
        method="GET",
        auth=auth,
    ).json()
    return search


def search_tweets(query: dict, auth: Union[OAuth1Auth, OAuth2Auth]):
    """Searches tweets via query (how to: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#build)

    :param query: Dict with all the arguments to build the query
    :param auth: OAuth object

    :return: Dict with the results
    """
    search = session.request(
        url=f"{tweets_url_v2}/search/recent?{urlencode(query)}",
        method="GET",
        auth=auth,
    ).json()
    return search


class TweetAction(Enum):
    POST = 1
    DELETE = 2


def tweet(
        action: TweetAction,
        auth: OAuth1Auth,
        id: Optional[str] = None,
        data: Optional[dict] = None
):
    """Posts or Deletes Tweet

    :param action: TweetAction (either POST, or DELETE)
    :param auth: OAuth object (has to be user_auth)
    :param id: In case of delete, this is the tweet ID
    :param data: In case of post, this is the dictionary {"text": "tweet text"}

    :return: result
    """
    url = f"{tweets_url_v2}/{id}" \
        if id and (action == TweetAction.DELETE) \
        else tweets_url_v2

    r = session.request(
        url=url,
        method=action.name,
        json=data,
        auth=auth,
    ).json()
    return r


def get_user_id(
        user_name: str,
        user_auth: OAuth1Auth,
):
    url = f"{base_api_url_v2}/users/by/username/{user_name}"
    result = session.get(
        url=url,
        auth=user_auth
    )

    return result.json()["data"]["id"]


def get_user_followers(
        user_id: str,
        user_auth: OAuth1Auth,
        max_results: None,
):
    url = f"{base_api_url_v2}/users/{user_id}/followers"

    if max_results:
        url += f'?{urlencode({"max_results": max_results})}'

    result = session.get(
        url=url,
        auth=user_auth
    )
    return result.json()


def get_user_info(query: dict, auth: Union[OAuth1Auth, OAuth2Auth]):
    url = f"{base_api_url_v2}/users?{urlencode(query)}"
    print(url)
    search = session.request(
        url=url,
        method="GET",
        auth=auth,
    ).json()
    return search

Ahora podemos, por ejemplo, buscar un Tweet por su ID:

In [49]:
id_list = ["1261326399320715264"]
tweets = get_tweets(ids=id_list, auth=app_auth)
tweets

{'data': [{'id': '1261326399320715264',
   'text': 'Tune in to the @MongoDB @Twitch stream featuring our very own @suhemparack to learn about Twitter Developer Labs - starting now! https://t.co/fAWpYi3o5O'}]}

O hacer búsquedas avanzadas por texto:

In [50]:
# Example of a more complex query:
query_text = {"query": '"learn python" lang:en',
              "tweet.fields": "id,author_id,geo,lang,public_metrics",
              "max_results": 15}
tweets = search_tweets(query=query_text, auth=app_auth)
tweets

{'data': [{'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1488631698602397699',
   'text': 'How you can learn Python with this 11 part series\nhttps://t.co/kZSS7KNlYr #Microsoft',
   'author_id': '1039406095976955905',
   'lang': 'en'},
  {'public_metrics': {'retweet_count': 3,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1488631178131353602',
   'text': 'RT @pythonbot_: Learn Python 3 the Hard Way: A Very Simple Introduction to the Terrifyingly Beautiful World of Computers and Code (Zed Shaw…',
   'author_id': '2149288499',
   'lang': 'en'},
  {'public_metrics': {'retweet_count': 1,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1488630182256603139',
   'text': 'RT @EverythingMS: How you can learn Python with this 11 part series https://t.co/w3ZNFHKgRr',
   'author_id': '1135684047500267520',
   'lang': 'en'},
  {'public_metrics': {'retweet_count': 0,
    're

Publicar un Tweet:

In [51]:
tweet_post = {"text": "hello world!"}

# create tweet:
publish_response = tweet(
    action=TweetAction.POST,
    data=tweet_post,
    auth=user_auth,
)

publish_response

{'data': {'id': '1488632885364076550', 'text': 'hello world!'}}

O borrarlo:

In [52]:
tweet_id = publish_response["data"]["id"]
delete_response = tweet(
    action=TweetAction.DELETE,
    id=tweet_id,
    auth=user_auth,
)
delete_response

{'data': {'deleted': True}}

También podemos extraer los *followers*

In [53]:
user_id = get_user_id(user_name="RafaelNadal", user_auth=user_auth)
user_id

'344634424'

In [54]:
followers = get_user_followers(user_id, user_auth, max_results=20)
followers

{'data': [{'id': '820097564', 'name': 'bethb', 'username': 'ekb979'},
  {'id': '1488631190621986820', 'name': 'George Pap', 'username': 'papant08'},
  {'id': '1488628566510215174', 'name': 'María', 'username': 'Mara068732332'},
  {'id': '1488160200884428811',
   'name': 'Cameron',
   'username': 'CameronJames288'},
  {'id': '1488631077350649861',
   'name': 'Dragutin Ivančić',
   'username': 'DragutinIvancic'},
  {'id': '1403730572', 'name': 'Wes Wiseman', 'username': 'W3isme'},
  {'id': '367130843', 'name': 'Tobias Mann', 'username': 'tobibrian'},
  {'id': '1487913964910649347',
   'name': 'Daniela Torres',
   'username': 'torresdanielahn'},
  {'id': '1464328159411720201',
   'name': 'Jaime Gracia',
   'username': '2jaimeles'},
  {'id': '813350528', 'name': 'Mauricio Jacquet', 'username': 'mauricej76'},
  {'id': '1480219223800659972',
   'name': 'Ana maria Zambrana Arze',
   'username': 'Annmary038'},
  {'id': '1347213191181455365',
   'name': 'Fernando Bartivas',
   'username': 'Fern

In [55]:
query_user = {
    "ids": user_id,
    "user.fields": "id,public_metrics,created_at,description,entities,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,withheld"
}

user_profile = get_user_info(query_user, user_auth)
user_profile


https://api.twitter.com/2/users?ids=344634424&user.fields=id%2Cpublic_metrics%2Ccreated_at%2Cdescription%2Centities%2Clocation%2Cname%2Cpinned_tweet_id%2Cprofile_image_url%2Cprotected%2Curl%2Cusername%2Cverified%2Cwithheld


{'data': [{'username': 'RafaelNadal',
   'pinned_tweet_id': '1469694125238857728',
   'verified': True,
   'name': 'Rafa Nadal',
   'id': '344634424',
   'profile_image_url': 'https://pbs.twimg.com/profile_images/898280220037443585/mthp5TlW_normal.jpg',
   'protected': False,
   'location': 'Manacor',
   'public_metrics': {'followers_count': 15673388,
    'following_count': 136,
    'tweet_count': 3127,
    'listed_count': 22697},
   'description': 'Tennis player',
   'url': '',
   'created_at': '2011-07-29T10:44:02.000Z'}]}